<a href="https://colab.research.google.com/github/Sahab00/Ann_Price_prediction_project/blob/main/Ann_price_prediction_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
data = pd.read_csv('USD_PKR_final_processed.csv')  # Replace 'your_file_name.csv' with your actual file name
print(data.head())


Saving USD_PKR_final_processed.csv to USD_PKR_final_processed (1).csv
         Date  Price   Open   High    Low  Change %   7_day_MA  14_day_MA  \
0  26/11/2007  61.32  61.12  61.36  61.02      0.30  61.268571  61.258214   
1  27/11/2007  61.32  61.22  61.40  61.17      0.00  61.268571  61.258214   
2  28/11/2007  61.11  61.22  61.40  60.98     -0.34  61.268571  61.258214   
3  29/11/2007  61.15  61.01  61.25  60.87      0.07  61.268571  61.258214   
4  30/11/2007  61.25  61.05  61.36  61.05      0.16  61.268571  61.258214   

   Rolling_Std  Price_Change  Lag_1  Lag_2  
0     0.113934      0.000000  61.32  61.32  
1     0.113934      0.000000  61.32  61.32  
2     0.113934     -0.342466  61.32  61.32  
3     0.113934      0.065456  61.11  61.32  
4     0.113934      0.163532  61.15  61.11  


In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Step 1: Load Data
# Replace 'exchange_rate.csv' with your dataset file.
data = pd.read_csv('USD_PKR_final_processed.csv')
print(data.head())

# Assume the dataset has 'Date' and 'Rate' columns.
rates = data['Price'].values

# Step 2: Data Preprocessing
scaler = MinMaxScaler(feature_range=(0, 1))
rates_scaled = scaler.fit_transform(rates.reshape(-1, 1))

sequence_length = 30  # Last 30 days' rates
X, Y = [], []
for i in range(len(rates_scaled) - sequence_length):
    X.append(rates_scaled[i:i + sequence_length])
    Y.append(rates_scaled[i + sequence_length])

X = np.array(X)
Y = np.array(Y)

# Reshape for LSTM input
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Step 3: Build LSTM Model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(sequence_length, 1)),
    LSTM(50),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
print(model.summary())

# Step 4: Train Model
model.fit(X, Y, epochs=20, batch_size=32, verbose=1)

# Step 5: Predict Future Values
last_sequence = rates_scaled[-sequence_length:]  # Use last 30 days
last_sequence = np.reshape(last_sequence, (1, sequence_length, 1))
predicted_rate = model.predict(last_sequence)
predicted_rate_original = scaler.inverse_transform(predicted_rate)

n_days_ahead = int(input("Enter the number of days ahead to predict: "))

print(f"Predicted Dollar to PKR rate ({n_days_ahead} days ahead): {predicted_rate_original[0][0]}")


#print("Predicted Dollar to PKR rate:", predicted_rate_original[0][0])


         Date  Price   Open   High    Low  Change %   7_day_MA  14_day_MA  \
0  26/11/2007  61.32  61.12  61.36  61.02      0.30  61.268571  61.258214   
1  27/11/2007  61.32  61.22  61.40  61.17      0.00  61.268571  61.258214   
2  28/11/2007  61.11  61.22  61.40  60.98     -0.34  61.268571  61.258214   
3  29/11/2007  61.15  61.01  61.25  60.87      0.07  61.268571  61.258214   
4  30/11/2007  61.25  61.05  61.36  61.05      0.16  61.268571  61.258214   

   Rolling_Std  Price_Change  Lag_1  Lag_2  
0     0.113934      0.000000  61.32  61.32  
1     0.113934      0.000000  61.32  61.32  
2     0.113934     -0.342466  61.32  61.32  
3     0.113934      0.065456  61.11  61.32  
4     0.113934      0.163532  61.15  61.11  


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                        │ (None, 30, 50)              │          10,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 30,651 (119.73 KB)

 Trainable params: 30,651 (119.73 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - loss: 0.0141
Epoch 2/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 9.0858e-05
Epoch 3/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - loss: 1.1146e-04
Epoch 4/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - loss: 1.1748e-04
Epoch 5/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - loss: 1.0694e-04
Epoch 6/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 9s 31ms/step - loss: 1.1172e-04
Epoch 7/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 1.1349e-04
Epoch 8/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - loss: 1.0620e-04
Epoch 9/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - loss: 9.1687e-05
Epoch 10/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - loss: 8.0083e-05
Epoch 11/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 1.3220e-04
Epoch 12/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 6.8208e-05
Epoch 13/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - loss: 7.0310e-05
Epoch 14/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - lo